In [ ]:
import sys
sys.path.append('/usr/local/lib/python2.7/dist-packages')
sys.path.append('/usr/local/lib/python3.5/dist-packages')
sys.path

In [ ]:
import create_dict
import torch
import os
import json
import utils
import numpy as np

# Clean Data

In [7]:
#folder_name = '/data2'
#create_dict.clean_obj(folder_name)

# Generate Dataset

In [ ]:
#!pip3 freeze | grep PyWavefront

In [ ]:
#import pywavefront
#mesh2 = pywavefront.Wavefront('/home/.schramm/Documents/3d_retrieval/data/02747177/8bdea15ae5d929d0a2eb129d649f68cf/models/model_normalized.obj', collect_faces='True')


In [ ]:
input_folder = '/data'
suffix = '_test_gpu2' #suffix will append data_trainxxxx.json
max_elements_per_class = 5
create_dict.create_dictionary(input_folder, max_elements_per_class, suffix)

success: 91db88d4d0956e92a9a12c50dc330a66 03211117 # 1
success: eb2b450565c77e3efc7ff8f848d1c6c9 03211117 # 2
success: 871dcd6740ed8d2891a7ac029a825f73 03211117 # 3
success: aa73f1b28817376c6cf19e722198d69 03211117 # 4
success: 4516c31dae48a0efbff4c0ca0d98b77 03211117 # 5
success: f4a259b4778a77b37b2bb75885cfc44 03211117 # 6
success: cd71c33773db5570be7e507ed67c0331 03211117 # 7
success: 64c23d38d98f7529811348287b48cb45 03211117 # 8
success: 2b0b8c07582c83d426bb7052f66b6d5b 03211117 # 9
success: d53e94b52e0cd47f71dd5d95c394249c 03211117 # 10
success: 94482409d0396dfa3f8f528a267748f 03211117 # 11
success: 90d97637c01fd05e59122da3e1ccb92c 03211117 # 12
success: fac8bc5420f9d472b36c473722d8f02c 03211117 # 13
success: 36ea450f5d479481c795085b4d203932 03211117 # 14
success: 5286df5fdb7ab5cf403c2939fca0114 03211117 # 15
success: bb2639fdd7ee85202f58405e4baaa2ed 03211117 # 16
success: f2f41913123a7a7e4b87db09ac4cfa73 03211117 # 17
success: 5fb06d76ec44a16bfa6dbbeb09c2f316 03211117 # 18
succes

success: bb1f8f4efd67582aa4aa57d4816c68a3 02818832 # 3
success: 1ef20a206e940a4bd79d7db7b8ed51c8 02818832 # 4
success: a8cfed0a1b9451c9d213467246722099 02818832 # 5
success: c758919a1fbe3d0c9cef17528faf7bc5 02818832 # 6
success: b6ac23d327248d72627bb9f102840372 02818832 # 7
success: 97f9d6942dfc12c08880edc6251fa529 02818832 # 8
success: a7d64e6a7223641733b13410d81359e4 02818832 # 9
success: ce0f3c9d6a0b0cda71010004e0594e66 02818832 # 10
success: d8de84e7c0f8cbb19473aab1da686220 02818832 # 11
success: 5f9dd5306ad6b3539867b7eda2e4d345 02818832 # 12
success: 1f11b3d9953fabcf8b4396b18c85cf0f 02818832 # 13
success: d3aa611e639ea0228998b3b64a143d42 02818832 # 14
success: c7f039115a83939ebbc004a1c4eccc8 02818832 # 15
success: 9daa02ae72e74b3f6bb2e9a62ccd177a 02818832 # 16
success: 560854f624e60b6cadab4c26397edfab 02818832 # 17
success: 30eb0a2f835b91b19e9c6b0794a3b4d8 02818832 # 18
success: 3fc3087faa29bce5b1b6b0c5ddacb607 02818832 # 19
success: 8a3240bc541767544b44261aa73cb2d9 02818832 # 20


# Initialize Network

In [ ]:
import SiameseNet

device = torch.device("cuda:0" if torch.cuda.torch.cuda.is_available() else "cpu")

batch_size = 2
net = SiameseNet.SiameseNet(batch_size)
suffix = '_gpu' # comment in if not coming from generating the dataset
path_to_params = "models/allClasses500_300epochs_m1_LR1e5.pt" # if file does not exist or is empty it starts from untrained and later saves to the file

# shift to GPU if available

net.to(device)
working_dir = os.getcwd()
data_dir_train = os.path.join(working_dir, 'data_train'+suffix+'.json')
data_dir_val = os.path.join(working_dir, 'data_val'+suffix+'.json')
class_dir = os.path.join(working_dir, 'class_dict'+suffix+'.json')

if os.path.isfile(path_to_params):
    if os.stat(path_to_params).st_size != 0:
        net.load_state_dict(torch.load(path_to_params))  #Loads pretrained net if file exists and if not empty
else:
    open(path_to_params, "x") #Creates parameter file if it does not exist

RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location='cpu' to map your storages to the CPU.

In [ ]:
device

# Train Network

In [ ]:
#training parameters

writer_suffix = 'testing'
margin = 0.5
num_epochs = 1
print_batch = 1
lr = 0.0001



In [ ]:
net = SiameseNet.train(net, num_epochs, margin, lr, print_batch, 
                       data_dir_train, data_dir_val, writer_suffix, path_to_params, working_dir)

# Validation

In [ ]:
margin = 0.5
writer_suffix = 'Val'
SiameseNet.val(net, margin, data_dir_val, writer_suffix, working_dir, class_dir)

# Retrieval

In [ ]:
y_true, y_pred, ids, shape, description = SiameseNet.retrieval(net, data_dir_val, working_dir, print_nn=False)

In [ ]:
num_KNN = 4
max_show = 3
utils.retrieve_images(y_pred, ids, data_dir_val, class_dir, num_KNN, max_show, shuffle=True)

In [ ]:
sentence = 'This is a black chair'
#sentence = input("Enter sentence")
num_KNN = 4
utils.retrieve_one_sentence(net, data_dir_val, working_dir, sentence, class_dir, num_KNN)
